<a href="https://colab.research.google.com/github/TamHoaVo/ML-Project/blob/main/DMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random

# additional libraries for data processing, visualization, and metric calculation
from sklearn.model_selection import train_test_split
from itertools import product
import pandas as pd
from scipy.stats import entropy
from IPython.display import display
import matplotlib.pyplot as plt
from collections import defaultdict
import copy
from scipy.spatial.distance import euclidean
from scipy.special import kl_div

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
# essentially random seeds help ensure that everytime we run code, you get the same results
# Use (cuda) GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Data Loading and Preprocessing
# Load training and test files
train_df = pd.read_csv('train2.csv')
test_df = pd.read_csv('test2.csv')
# Rename 'item_id' column to 'item' in the training data
train_df = train_df.rename(columns={'item_id': 'item'})
# Rename 'movieId' column to 'item' in the test data so both datasets have the same column name
test_df = test_df.rename(columns={'movieId': 'item'})
# Combine training and test data into one DataFrame
full_df = pd.concat([train_df, test_df], ignore_index=True)


# Convert original user and item identifiers to numerical indices for model compatibility
# Also, transform the rating score into a binary classification label (1 if rating >= 4, else 0)
user_mapping = {u: idx for idx, u in enumerate(full_df['tag'].unique())}
item_mapping = {i: idx for idx, i in enumerate(full_df['item'].unique())}

full_df['user_id'] = full_df['tag'].map(user_mapping)
full_df['item_id'] = full_df['item'].map(item_mapping)
full_df['label'] = (full_df['targets'] >= 4).astype(int)

sample_df = full_df.sample(n=10000, random_state=42)

# Create a list of (user_id, item_id, label) interactions from the sampled DataFrame
interactions = list(zip(sample_df['user_id'], sample_df['item_id'], sample_df['label']))


# Define a function to flip binary labels (1 -> 0, 0 -> 1) for targeted users
# This simulates label modification for unlearning or adversarial scenarios
def flip_user_labels(interactions, target_users):
    return [(u, i, 1 - l) if u in target_users else (u, i, l) for u, i, l in interactions]

def compute_fliprec_loss(preds, labels, teacher_preds, users, target_users, lambda_general=0.5, lambda_target=0.0):
    bce = nn.BCELoss(reduction='none')(preds, labels)
    mse = F.mse_loss(preds, teacher_preds, reduction='none')
    is_target = torch.tensor([u.item() in target_users for u in users], device=preds.device, dtype=torch.float)
    lambdas = is_target * lambda_target + (1 - is_target) * lambda_general
    return (bce + lambdas * mse).mean()

# - A small fraction (e.g., 5%) as targeted users (for unlearning or manipulation)
# - The rest as retained users (used to evaluate stability or preservation)
def split_users(interactions, target_fraction=0.05):
    users = list(set(u for u, _, _ in interactions))
    np.random.shuffle(users)
    cutoff = int(len(users) * target_fraction)
    return set(users[:cutoff]), set(users[cutoff:])

# Ensure non-empty targeted and retained groups
target_users, retained_users = split_users(interactions)
sample_df['user_group'] = sample_df['user_id'].apply(lambda u: 'Targeted' if u in target_users else 'Retained')

#separate the dataset into two data frames: one for retained users and the other for targeted users
retained_df = sample_df[sample_df['user_group'] == 'Retained']
targeted_df = sample_df[sample_df['user_group'] == 'Targeted']

# convert each group's DataFrame into a list of user, item, label interactions
retained_interactions = list(zip(retained_df['user_id'], retained_df['item_id'], retained_df['label']))
targeted_interactions = list(zip(targeted_df['user_id'], targeted_df['item_id'], targeted_df['label']))
all_interactions = retained_interactions + targeted_interactions

# determines total number of users and items that are used for model input dimensions
num_users = sample_df['user_id'].max() + 1
num_items = sample_df['item_id'].max() + 1

# Print statistics about the split and data dimensions
print(f"Retained Users: {len(retained_users)}, Targeted Users: {len(target_users)}")
print(f"Retained Interactions: {len(retained_interactions)}, Targeted Interactions: {len(targeted_interactions)}")
print(f"Number of users: {num_users}, Number of items: {num_items}")
print(f"Total interactions: {len(interactions)}")

# Function to compute AUC (Area Under the ROC Curve) for binary classification performance
def calculate_auc(predictions, labels):
    from sklearn.metrics import roc_auc_score
    return roc_auc_score(labels, predictions)

# Function to compute Hit Rate @k:
# For each user, check if a relevant (label==1) item appears in their top-k predicted items
def calculate_hr(predictions, labels, users, items, k=10):
    user_item_scores = defaultdict(list)
    for u, i, p, l in zip(users, items, predictions, labels):
        user_item_scores[u].append((p, l))
    hits = 0
    total = 0
    for user, scores in user_item_scores.items():
        if len(scores) < k:
            continue
        sorted_scores = sorted(scores, key=lambda x: x[0], reverse=True)[:k]
        if any(label == 1 for _, label in sorted_scores):
            hits += 1
        total += 1
    return hits / total if total > 0 else 0

def calculate_ndcg(predictions, labels, users, items, k=10):
    user_item_scores = defaultdict(list)
    for u, i, p, l in zip(users, items, predictions, labels):
        user_item_scores[u].append((p, l))
    ndcg_total = 0
    total = 0
    for user, scores in user_item_scores.items():
        if len(scores) < k:
            continue
        sorted_scores = sorted(scores, key=lambda x: x[0], reverse=True)[:k]
        ideal_scores = sorted(scores, key=lambda x: x[1], reverse=True)[:k]
        dcg = sum([l / np.log2(idx + 2) for idx, (_, l) in enumerate(sorted_scores)])
        idcg = sum([l / np.log2(idx + 2) for idx, (_, l) in enumerate(ideal_scores)])
        if idcg > 0:
            ndcg_total += dcg / idcg
            total += 1
    return ndcg_total / total if total > 0 else 0

# DMF Model Definition
class DMF(nn.Module):
    def __init__(self, num_users, num_items):
        super(DMF, self).__init__()

        # User tower
        # Transforms user indices into 512-dimensional vectors
        # Processes the user embedding through a 3-layer MLP:512 → 256 → 128 → 64
        self.user_embed = nn.Embedding(num_users, 512)
        self.user_mlp = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )

        # Item tower
        # Transforms item indices into 1024-dimensional vectors
        # Processes the item embedding through a 4-layer MLP: 1024 → 512 → 256 → 128 → 64
        self.item_embed = nn.Embedding(num_items, 1024)
        self.item_mlp = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )

        # Final prediction (cosine similarity)
        self.sigmoid = nn.Sigmoid()

    def forward(self, user, item):
        #  Get the embeddings
        user_vec = self.user_embed(user)
        item_vec = self.item_embed(item)
        # Process embeddings through each tower
        user_out = self.user_mlp(user_vec)
        item_out = self.item_mlp(item_vec)

        # Cosine similarity as final prediction
        # Compute similarity between processed vectors
        sim = F.cosine_similarity(user_out, item_out)
        # Squash it between 0 and 1 for final prediction
        return self.sigmoid(sim.unsqueeze(1))


class InteractionDataset(Dataset):
    def __init__(self, user_item_label, num_users, num_items):
        self.data = user_item_label
        self.num_users = num_users
        self.num_items = num_items

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        user, item, label = self.data[idx]
        user = min(user, self.num_users - 1)
        item = min(item, self.num_items - 1)
        return torch.tensor(user, dtype=torch.long), torch.tensor(item, dtype=torch.long), torch.tensor(label, dtype=torch.float)

def train_model_with_params(model, original_model, train_data, target_users, device, lr, bs, lambda_general=0.5, num_epochs=1):
    if len(train_data) == 0:
      raise ValueError("Training data is empty. Cannot proceed with training.")

    loader = DataLoader(
        InteractionDataset(train_data, model.user_embed.num_embeddings, model.item_embed.num_embeddings),
        batch_size=bs,
        shuffle=True
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    original_model.to(device)
    original_model.eval()
    metric_results = {"AUC": [], "HR@5": [], "HR@10": [], "HR@20": [], "NDCG@5": [], "NDCG@10": [], "NDCG@20": []}

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        predictions = []
        true_labels = []
        user_list = []
        item_list = []

        for users, items, labels in loader:
            users, items, labels = users.to(device), items.to(device), labels.to(device)
            optimizer.zero_grad()
            preds = model(users, items).squeeze()
            with torch.no_grad():
                teacher_preds = original_model(users, items).squeeze()
            loss = compute_fliprec_loss(preds, labels, teacher_preds, users, target_users, lambda_general)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predictions.extend(preds.cpu().detach().numpy())
            true_labels.extend(labels.cpu().detach().numpy())
            user_list.extend(users.cpu().numpy())
            item_list.extend(items.cpu().numpy())

        auc = calculate_auc(predictions, true_labels)
        hr5 = calculate_hr(predictions, true_labels, user_list, item_list, k=5)
        hr10 = calculate_hr(predictions, true_labels, user_list, item_list, k=10)
        hr20 = calculate_hr(predictions, true_labels, user_list, item_list, k=20)
        ndcg5 = calculate_ndcg(predictions, true_labels, user_list, item_list, k=5)
        ndcg10 = calculate_ndcg(predictions, true_labels, user_list, item_list, k=10)
        ndcg20 = calculate_ndcg(predictions, true_labels, user_list, item_list, k=20)

        metric_results["AUC"].append(auc)
        metric_results["HR@5"].append(hr5)
        metric_results["HR@10"].append(hr10)
        metric_results["HR@20"].append(hr20)
        metric_results["NDCG@5"].append(ndcg5)
        metric_results["NDCG@10"].append(ndcg10)
        metric_results["NDCG@20"].append(ndcg20)

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}, AUC: {auc:.4f}, HR@5: {hr5:.4f}, HR@10: {hr10:.4f}, HR@20: {hr20:.4f}, NDCG@5: {ndcg5:.4f}, NDCG@10: {ndcg10:.4f}, NDCG@20: {ndcg20:.4f}")

    return metric_results

# Unlearning Methods Implementation
def retrained_baseline(interactions, num_users, num_items, target_users, lr, bs, device, num_epochs=1):
    """Retrains the model from scratch after removing targeted user data (only when training on retained users)."""

    # Do NOT filter if we're already training on targeted users only
    user_ids = set(u for u, _, _ in interactions)
    if user_ids.issubset(target_users):
        remaining_data = interactions
    else:
        remaining_data = [(u, i, l) for u, i, l in interactions if u not in target_users]

    if len(remaining_data) == 0:
        raise ValueError("Training data is empty. Cannot proceed with training.")

    model = DMF(num_users, num_items)
    original_model = copy.deepcopy(model)
    return train_model_with_params(model, original_model, remaining_data, target_users, device, lr, bs, lambda_general=0.5, num_epochs=num_epochs)


def sisa(interactions, num_users, num_items, target_users, lr, bs, device, num_epochs=1, num_shards=5):
    """Implements the SISA method with submodel training."""
    random.shuffle(interactions)
    shards = [interactions[i::num_shards] for i in range(num_shards)]
    submodels = []
    for shard in shards:
        model = DMF(num_users, num_items)
        original_model = copy.deepcopy(model)
        submodels.append(train_model_with_params(model, original_model, shard, target_users, device, lr, bs, lambda_general=0.5, num_epochs=num_epochs))
    return submodels

def receraser(interactions, num_users, num_items, target_users, lr, bs, device, num_epochs=1, num_shards=5):
    """Implements RecEraser by grouping shards based on user-item similarity."""
    interactions_by_user = {}
    for u, i, l in interactions:
        interactions_by_user.setdefault(u, []).append((i, l))

    shards = [list(interactions_by_user.keys())[i::num_shards] for i in range(num_shards)]
    submodels = []
    for shard in shards:
        shard_data = [(u, i, l) for u in shard for i, l in interactions_by_user[u]]
        model = DMF(num_users, num_items)
        original_model = copy.deepcopy(model)
        submodels.append(train_model_with_params(
            model=model,
            original_model=original_model,
            train_data=shard_data,
            target_users=target_users,
            device=device,
            lr=lr,
            bs=bs,
            lambda_general=0.5,
            num_epochs=num_epochs
        ))
    return submodels

def badt(interactions, num_users, num_items, target_users, lr, bs, device, num_epochs=1):
    """Implements the BadT method using student-teacher models."""
    teacher_model = DMF(num_users, num_items)
    teacher_model.eval()
    student_model = DMF(num_users, num_items)
    optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
    student_model.to(device)
    teacher_model.to(device)

    for epoch in range(num_epochs):
        student_model.train()
        loader = DataLoader(
            InteractionDataset(interactions, num_users, num_items),
            batch_size=bs,
            shuffle=True
        )
        for users, items, labels in loader:
            users, items, labels = users.to(device), items.to(device), labels.to(device)
            optimizer.zero_grad()

            student_preds = student_model(users, items).squeeze()
            with torch.no_grad():
                teacher_preds = teacher_model(users, items).squeeze()

            loss = compute_fliprec_loss(student_preds, labels, teacher_preds, users, target_users)
            loss.backward()
            optimizer.step()

    # Do NOT filter out targeted users if we're evaluating on them
    user_ids = set(u for u, _, _ in interactions)
    if user_ids.issubset(target_users):
        final_data = interactions
    else:
        final_data = [(u, i, l) for u, i, l in interactions if u not in target_users]

    if len(final_data) == 0:
        raise ValueError("Training data is empty. Cannot proceed with training.")

    return train_model_with_params(
        model=student_model,
        original_model=student_model,
        train_data=final_data,
        target_users=set(),
        device=device,
        lr=lr,
        bs=bs,
        lambda_general=0.5,
        num_epochs=num_epochs
    )


def ermax(interactions, num_users, num_items, target_users, lr, bs, device, num_epochs=1):
    """Implements the erMax method to maximize errors on targeted users."""

    # Step 1: Add noise by flipping labels for targeted users
    noisy_data = [(u, i, random.randint(0, 1)) if u in target_users else (u, i, l) for u, i, l in interactions]

    model = DMF(num_users, num_items)
    original_model = copy.deepcopy(model)

    # Phase 1: Train with noise
    train_model_with_params(
        model=model,
        original_model=original_model,
        train_data=noisy_data,
        target_users=target_users,
        device=device,
        lr=lr,
        bs=bs,
        lambda_general=1.0,
        num_epochs=num_epochs
    )

    # Step 2: Prepare clean data for fine-tuning
    user_ids = set(u for u, _, _ in interactions)

    if user_ids.issubset(target_users):
        cleaned_data = interactions  # If all are targeted, do not filter
    else:
        cleaned_data = [(u, i, l) for u, i, l in interactions if u not in target_users]

    if len(cleaned_data) == 0:
        raise ValueError("Training data is empty after filtering targeted users.")

    return train_model_with_params(
        model=model,
        original_model=model,
        train_data=cleaned_data,
        target_users=set(),
        device=device,
        lr=lr,
        bs=bs,
        lambda_general=0.5,
        num_epochs=num_epochs
    )


def fliprec(interactions, num_users, num_items, target_users, lr, bs, device, lambda_general=0.5, lambda_target=0.0, num_epochs=1):
    """Train FlipRec using student-teacher framework with per-user loss adjustment."""

    # Step 1: Label flip for targeted users
    flipped_data = [(u, i, 1 - l) if u in target_users else (u, i, l) for u, i, l in interactions]

    # Step 2: Contextual User Augmentation (Dtrs)
    item_to_targeted_users = defaultdict(set)
    user_item_pos = defaultdict(set)

    for u, i, l in interactions:
        if u in target_users:
            item_to_targeted_users[i].add(u)
        if l == 1:
            user_item_pos[u].add(i)

    contextual_users = set()
    for i in item_to_targeted_users:
        for u in user_item_pos:
            if u not in target_users and i in user_item_pos[u]:
                contextual_users.add(u)

    contextual_data = []
    for u in contextual_users:
        shared_items = [i for i in user_item_pos[u] if i in item_to_targeted_users]
        positives = [(u, i, 1) for i in shared_items]
        negatives = [(u, random.randint(0, num_items - 1), 0) for _ in shared_items]
        contextual_data.extend(positives + negatives)

    # Step 3: Combine Dflipf + Dtrs
    final_data = flipped_data + contextual_data

    # Step 4: Setup model and teacher
    model = DMF(num_users, num_items)
    teacher = copy.deepcopy(model)

    # Step 5: Train using modified train function with KD support
    return train_model_with_params(
        model=model,
        original_model=teacher,
        train_data=final_data,
        target_users=target_users,
        device=device,
        lr=lr,
        bs=bs,
        lambda_general=lambda_general,
        num_epochs=num_epochs
    )



# Prepare separate result holders
all_results_ret = []
all_results_tgt = []

# Evaluate each method on both retained and targeted interactions
methods = [
    ("Retrained", retrained_baseline),
    ("SISA", sisa),
    ("RecEraser", receraser),
    ("BadT", badt),
    ("erMax", ermax),
    ("FlipRec", fliprec)
]

for method_name, method_func in methods:
    print(f"Running {method_name} on Retained users...")
    method_results_ret = method_func(
        interactions=retained_interactions,
        num_users=num_users,
        num_items=num_items,
        target_users=target_users,
        lr=0.00015,
        bs=64,
        device=device
    )
    all_results_ret.append((method_name, method_results_ret))

    print(f"Running {method_name} on Targeted users...")
    method_results_tgt = method_func(
        interactions=targeted_interactions,
        num_users=num_users,
        num_items=num_items,
        target_users=target_users,
        lr=0.00015,
        bs=64,
        device=device
    )
    all_results_tgt.append((method_name, method_results_tgt))

# Collect Results in a DataFrame (split by group)
def collect_groupwise_results(methods, all_results_ret, all_results_tgt):
    retained_data = []
    targeted_data = []
    for i, (method_name, _) in enumerate(methods):
        results_ret = all_results_ret[i][1]
        results_tgt = all_results_tgt[i][1]

        def extract_metrics(results):
            if isinstance(results, list):
                return [
                    np.mean([res[metric][-1] for res in results])
                    for metric in ["AUC", "HR@5", "HR@10", "HR@20", "NDCG@5", "NDCG@10", "NDCG@20"]
                ]
            else:
                return [
                    results["AUC"][-1], results["HR@5"][-1], results["HR@10"][-1],
                    results["HR@20"][-1], results["NDCG@5"][-1], results["NDCG@10"][-1], results["NDCG@20"][-1]
                ]

        retained_metrics = extract_metrics(results_ret)
        targeted_metrics = extract_metrics(results_tgt)

        retained_data.append([method_name] + retained_metrics)
        targeted_data.append([method_name] + targeted_metrics)

    columns = ["Method", "AUC", "HR@5", "HR@10", "HR@20", "NDCG@5", "NDCG@10", "NDCG@20"]
    df_retained = pd.DataFrame(retained_data, columns=columns)
    df_targeted = pd.DataFrame(targeted_data, columns=columns)
    print("\nRetained Results Summary:")
    display(df_retained)
    print("\nTargeted Results Summary:")
    display(df_targeted)
    return df_retained, df_targeted

# Generate and display the groupwise results
df_retained, df_targeted = collect_groupwise_results(methods, all_results_ret, all_results_tgt)

# Activation Distance & JS-Divergence Calculation
def calculate_activation_distance_and_js(original_probs, unlearned_probs, users, target_users):
    def safe_probs(p):
        p = np.clip(np.array(p), 1e-9, 1.0)
        return p / np.sum(p)

    distances = {"Targeted": [], "Retained": [], "All": []}
    js_values = {"Targeted": [], "Retained": [], "All": []}

    for p1, p2, u in zip(original_probs, unlearned_preds, users):
        p1 = safe_probs([p1[0], 1 - p1[0]])
        p2 = safe_probs([p2[0], 1 - p2[0]])
        m = 0.5 * (p1 + p2)
        js = 0.5 * (np.sum(kl_div(p1, m)) + np.sum(kl_div(p2, m)))
        dist = euclidean(p1, p2)

        group = "Targeted" if u in target_users else "Retained"
        distances[group].append(dist)
        js_values[group].append(js)
        distances["All"].append(dist)
        js_values["All"].append(js)

    return {
        "Activation": {g: np.mean(distances[g]) for g in distances},
        "JS": {g: np.mean(js_values[g]) for g in js_values},
    }

# Display as Table
def display_effectiveness_table(results_by_method):
    rows = []
    for group in ["Targeted", "Retained", "All"]:
        act_row = [group + " (Activation)"] + [f"{results_by_method[m]['Activation'][group]:.2f}" for m in results_by_method]
        js_row = [group + " (JS-Div)"] + [f"{results_by_method[m]['JS'][group]:.2f}" for m in results_by_method]
        rows.append(act_row)
        rows.append(js_row)
    methods = list(results_by_method.keys())
    df = pd.DataFrame(rows, columns=["Group"] + methods)
    print("Activation Distance & JS-Divergence Summary:")
    display(df)

# Run Activation Distance & JS-Divergence Evaluation
results_by_method = {}

# Use original model trained on full data as the baseline
baseline_model = DMF(num_users, num_items).to(device)
baseline_model.eval()
loader = DataLoader(InteractionDataset(interactions, num_users, num_items), batch_size=64)
original_preds = []
user_ids = []

with torch.no_grad():
    for users, items, _ in loader:
        users, items = users.to(device), items.to(device)
        probs = baseline_model(users, items).cpu().numpy()
        original_preds.extend(probs)
        user_ids.extend(users.cpu().numpy())

for method_name, method_func in methods:
    print(f"Evaluating {method_name} model for activation and JS...")
    model = DMF(num_users, num_items).to(device)
    model.eval()
    unlearned_preds = []

    with torch.no_grad():
        for users, items, _ in loader:
            users, items = users.to(device), items.to(device)
            probs = model(users, items).cpu().numpy()
            unlearned_preds.extend(probs)

    metrics = calculate_activation_distance_and_js(original_preds, unlearned_preds, user_ids, target_users)
    results_by_method[method_name] = metrics

display_effectiveness_table(results_by_method)

Using device: cuda
Retained Users: 996, Targeted Users: 52
Retained Interactions: 9331, Targeted Interactions: 669
Number of users: 1083, Number of items: 5032
Total interactions: 10000
Running Retrained on Retained users...
Epoch 1, Loss: 0.6759, AUC: 0.5430, HR@5: 0.8833, HR@10: 0.9794, HR@20: 0.9917, NDCG@5: 0.4846, NDCG@10: 0.5108, NDCG@20: 0.5282
Running Retrained on Targeted users...
Epoch 1, Loss: 0.6647, AUC: 0.5151, HR@5: 0.8571, HR@10: 0.9444, HR@20: 1.0000, NDCG@5: 0.4111, NDCG@10: 0.4202, NDCG@20: 0.4801
Running SISA on Retained users...
Epoch 1, Loss: 0.6826, AUC: 0.5178, HR@5: 0.9138, HR@10: 1.0000, HR@20: 1.0000, NDCG@5: 0.5742, NDCG@10: 0.5041, NDCG@20: 0.6703
Epoch 1, Loss: 0.6825, AUC: 0.5227, HR@5: 0.9630, HR@10: 1.0000, HR@20: 1.0000, NDCG@5: 0.5655, NDCG@10: 0.6024, NDCG@20: 0.7392
Epoch 1, Loss: 0.6861, AUC: 0.5055, HR@5: 0.9000, HR@10: 1.0000, HR@20: 1.0000, NDCG@5: 0.5211, NDCG@10: 0.5988, NDCG@20: 0.6818
Epoch 1, Loss: 0.6810, AUC: 0.5181, HR@5: 0.9444, HR@10: 

,Method,AUC,HR@5,HR@10,HR@20,NDCG@5,NDCG@10,NDCG@20
0,Retrained,0.542960,0.883275,0.979381,0.991736,0.484595,0.510826,0.528172
1,SISA,0.515382,0.922881,1.000000,1.000000,0.547722,0.576541,0.618857
2,RecEraser,0.523713,0.909696,0.990181,1.000000,0.504090,0.522812,0.539834
3,BadT,0.720954,0.944251,0.996564,1.000000,0.681518,0.706135,0.710859
4,erMax,0.702046,0.940767,0.989691,1.000000,0.670773,0.674698,0.689735
5,FlipRec,0.526567,0.878049,0.982818,1.000000,0.465662,0.494610,0.518520



Targeted Results Summary:


,Method,AUC,HR@5,HR@10,HR@20,NDCG@5,NDCG@10,NDCG@20
0,Retrained,0.515121,0.857143,0.944444,1.0,0.411145,0.420238,0.480081
1,SISA,0.524200,0.766667,0.933333,1.0,0.555243,0.545403,0.742159
2,RecEraser,0.536910,0.813333,0.950000,1.0,0.467925,0.458887,0.499887
3,BadT,0.760129,0.892857,0.944444,1.0,0.681684,0.689911,0.674871
4,erMax,0.560893,0.750000,0.944444,1.0,0.441869,0.449356,0.503623
5,FlipRec,0.528627,1.000000,1.000000,1.0,0.692820,0.668940,0.721476


Evaluating Retrained model for activation and JS...
Evaluating SISA model for activation and JS...
Evaluating RecEraser model for activation and JS...
Evaluating BadT model for activation and JS...
Evaluating erMax model for activation and JS...
Evaluating FlipRec model for activation and JS...
Activation Distance & JS-Divergence Summary:


,Group,Retrained,SISA,RecEraser,BadT,erMax,FlipRec
0,Targeted (Activation),0.03,0.05,0.04,0.04,0.06,0.04
1,Targeted (JS-Div),0.00,0.00,0.00,0.00,0.00,0.00
2,Retained (Activation),0.04,0.04,0.04,0.05,0.04,0.04
3,Retained (JS-Div),0.00,0.00,0.00,0.00,0.00,0.00
4,All (Activation),0.04,0.04,0.04,0.05,0.04,0.04
5,All (JS-Div),0.00,0.00,0.00,0.00,0.00,0.00
